In [1]:
from __future__ import print_function

import os
import re
import cPickle, gzip
import progressbar
from rdkit.Chem import AllChem
import parser.Smipar as Smipar

In [2]:
with gzip.open('gen_rxn/data/all_rxns.pkl.gz', 'rb') as f:
    rxn_all = cPickle.load(f)

data_length = len(rxn_all)
print(data_length)

865118


In [3]:
# data reloader

with gzip.open('data/vocab/vocab_dict.pkl.gz', 'rb') as dict_file:
    vocab_reactants, vocab_products = cPickle.load(dict_file)

with gzip.open('data/vocab/vocab_list.pkl.gz', 'rb') as list_file:
    reactants_token_list, products_token_list = cPickle.load(list_file)

In [4]:
bar = progressbar.ProgressBar(max_value=data_length)

token_len = []

with gzip.open('data/training_data(gen_rxn).pkl.gz', 'wb') as training_file:
    
    for i, rsmi in enumerate(rxn_all):

        reactant_list = []
        agent_list = []
        product_list = []

        split_rsmi = rsmi.split('>')
        reactants = split_rsmi[0].split('.')
        agents = split_rsmi[1].split('.')
        products = split_rsmi[2].split('.')

        for reactant in reactants:
            reactant_list += Smipar.parser_list(reactant)
            reactant_list += '.'
        for agent in agents:
            agent_list += Smipar.parser_list(agent)
            agent_list += '.'
        for product in products:
            product_list += Smipar.parser_list(product)
            product_list += '.'

        reactant_list.pop() # to pop last '.'
        agent_list.pop()
        product_list.pop()

        reactant_list += '>'
        reactant_list += agent_list
        
        token_len.append((len(reactant_list), len(product_list)))

        cPickle.dump(([reactants_token_list.index(r) for r in reactant_list],
                      [products_token_list.index(p) for p in product_list],
                      rsmi[1]), training_file, 2)
                     
        bar.update(i)

 99% (865117 of 865118) |################## | Elapsed Time: 1:35:27 ETA: 0:00:00

In [5]:
trans = zip(*token_len)
print(max(trans[0]), max(trans[1]))

75 47


In [9]:
count=0
bar = progressbar.ProgressBar(max_value=data_length)
with gzip.open('data/training_data(gen_rxn).pkl.gz', 'rb') as data_file:
    with gzip.open('data/train(gen_rxn).pkl.gz', 'wb') as train_file:
        with gzip.open('data/dev(gen_rxn).pkl.gz', 'wb') as dev_file:
            i = 1
            while 1:
                try:
                    reactants, products, _ = cPickle.load(data_file)
                    if i%100:
                        cPickle.dump((reactants, products), train_file, 2)
                    else:
                        cPickle.dump((reactants, products), dev_file, 2)
                        count += 1
                except EOFError:
                    break
                bar.update(i)
                i += 1

import time
time.sleep(1)

print(count)

 99% (864725 of 865118) |################## | Elapsed Time: 0:02:00 ETA: 0:00:00

8651
